In [1]:
import numpy as np

from skimage import io
import os
import napari

# import dask_image
import dask.array as da
import dask
import zarr

from fractions import Fraction
from skimage.transform import resize

/home/tmurakami/app/miniconda3/envs/segment/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
level = 2

imgs = {
    'R01_ch488':zarr.open('/mnt/ampa_data01/tmurakami/220806_visual_01_R01/ch488.zarr',mode='r')[level][:],
    'R01_ch561':zarr.open('/mnt/ampa_data01/tmurakami/220806_visual_01_R01/ch561.zarr',mode='r')[level][:],
    'R01_ch640':zarr.open('/mnt/ampa_data01/tmurakami/220806_visual_01_R01/ch640.zarr',mode='r')[level][:],
    'R02_ch488':zarr.open('/mnt/ampa_data01/tmurakami/220806_visual_01_R01/R02_R01/R02ch488_to_R01.zarr',mode='r')[level][:],
    'R02_ch640':zarr.open('/mnt/ampa_data01/tmurakami/220806_visual_01_R01/R02_R01/R02ch640_to_R01.zarr',mode='r')[level][:]
}

outpath = '/mnt/ampa_data01/tmurakami/220806_visual_01_R01/morphotrack'

In [3]:
voxel_size = (Fraction(3,1),Fraction(65,100),Fraction(65,100))
target_size = (10,10,10)

voxel_size_d = (2**level) * np.asarray(voxel_size)
fractions = voxel_size_d / np.asarray(target_size)

denominators = np.asarray([i.denominator for i in fractions])

In [4]:
# add dark voxel until I can resize the voxels precisely
shape = np.asarray(imgs['R01_ch488'].shape)
pad_target = np.ceil(shape / denominators).astype(int) * denominators

In [5]:
for key in imgs:
    img = imgs[key]
    pad_img = np.pad(img,tuple((0, i) for i in pad_target - shape), mode='constant', constant_values=0)
    img_adjusted = (resize(pad_img,tuple((pad_img.shape * fractions).astype(int)),preserve_range=True)).astype(np.uint16)
    io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)

/tmp/ipykernel_473047/235802711.py:5: UserWarning: /mnt/ampa_data01/tmurakami/220806_visual_01_R01/morphotrack/R01_ch561_10um.tif is a low contrast image
  io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)
/tmp/ipykernel_473047/235802711.py:5: UserWarning: /mnt/ampa_data01/tmurakami/220806_visual_01_R01/morphotrack/R01_ch640_10um.tif is a low contrast image
  io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)
/tmp/ipykernel_473047/235802711.py:5: UserWarning: /mnt/ampa_data01/tmurakami/220806_visual_01_R01/morphotrack/R02_ch640_10um.tif is a low contrast image
  io.imsave(os.path.join(outpath,key+'_10um.tif'), img_adjusted)


In [6]:
#